In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset, load_metric
import datasets
import torch
from nltk.translate.meteor_score import meteor_score
import sacrebleu
from bert_score import score

- Load Datasets

In [3]:
val_data = datasets.load_dataset("wmt16","de-en", split="validation")
test_data = datasets.load_dataset("wmt16","de-en", split="test")

print(len(val_data))
print(len(test_data))

2169
2999


- Load the T5 Model

In [5]:
# Load the T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [6]:
# Function to translate a sentence from German to English
def translate_sentence(sentence, model, tokenizer):
    input_text = f"translate German to English: {sentence}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

- Validation Set Experiments

In [8]:
# iterate over the validation data and translate the German sentences to English
val_references = []
val_translations = []

i = 0
for item in val_data:
    i += 1
    english_sentence = item["translation"]["en"]
    german_sentence = item["translation"]["de"]

    translated_sentence = translate_sentence(german_sentence, model, tokenizer)

    val_references.append(english_sentence)
    val_translations.append(translated_sentence)
    print(f"Translated {i} sentences")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Translated 1 sentences
Translated 2 sentences
Translated 3 sentences
Translated 4 sentences
Translated 5 sentences
Translated 6 sentences
Translated 7 sentences
Translated 8 sentences
Translated 9 sentences
Translated 10 sentences
Translated 11 sentences
Translated 12 sentences
Translated 13 sentences
Translated 14 sentences
Translated 15 sentences
Translated 16 sentences
Translated 17 sentences
Translated 18 sentences
Translated 19 sentences
Translated 20 sentences
Translated 21 sentences
Translated 22 sentences
Translated 23 sentences
Translated 24 sentences
Translated 25 sentences
Translated 26 sentences
Translated 27 sentences
Translated 28 sentences
Translated 29 sentences
Translated 30 sentences
Translated 31 sentences
Translated 32 sentences
Translated 33 sentences
Translated 34 sentences
Translated 35 sentences
Translated 36 sentences
Translated 37 sentences
Translated 38 sentences
Translated 39 sentences
Translated 40 sentences
Translated 41 sentences
Translated 42 sentences
T

In [9]:
print(val_references[:5])
print(val_translations[:5])

['India and Japan prime ministers meet in Tokyo', "India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.", 'Mr Modi is on a five-day trip to Japan to strengthen economic ties with the third largest economy in the world.', 'High on the agenda are plans for greater nuclear co-operation.', 'India is also reportedly hoping for a deal on defence collaboration between the two nations.']
['Die Premierministerin Indiens und Japans trafen sich in Tokio.', 'Der neue Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslands', 'Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die', 'Die Pläne für eine stärkere kerntechnische Zusammenarbeit stehen ganz oben auf der Tages', 'Berichten zufolge hofft Indien darüber hinaus auf einen Vertrag zur Verteidigung']


In [ ]:
# Iterate over the test data and translate the German sentences to English
test_references = []
test_translations = []

i = 0
for item in test_data:
    i += 1
    english_sentence = item["translation"]["en"]
    german_sentence = item["translation"]["de"]

    translated_sentence = translate_sentence(german_sentence, model, tokenizer)

    test_references.append(english_sentence)

    print(f"Translated {i} sentences")

In [ ]:
# Calculate BLEU score for all the translations
bleu_metric = load_metric("sacrebleu")

val_bleu_result = bleu_metric.compute(predictions=val_translations, references=val_references)
test_bleu_result = bleu_metric.compute(predictions=test_translations, references=test_references)

val_bleu_score = val_bleu_result["score"]
test_bleu_score = test_bleu_result["score"]

print(f"BLEU score for validation data: {val_bleu_score}")
print(f"BLEU score for test data: {test_bleu_score}")

In [ ]:
# Calculate METEOR score for all the translations
val_meteor_scores = []
test_meteor_scores = []

for i in range(len(val_references)):
    val_meteor_score = meteor_score([val_references[i]], val_translations[i])
    val_meteor_scores.append(val_meteor_score)

for i in range(len(test_references)):
    test_meteor_score = meteor_score([test_references[i]], test_translations[i])
    test_meteor_scores.append(test_meteor_score)

val_meteor_score = sum(val_meteor_scores) / len(val_meteor_scores)
test_meteor_score = sum(test_meteor_scores) / len(test_meteor_scores)

print(f"METEOR score for validation data: {val_meteor_score}")
print(f"METEOR score for test data: {test_meteor_score}")

In [ ]:
# BERTScore
val_bert_scores = score(val_translations, val_references, lang="en", verbose=True)